### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
from fpdf import FPDF
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 350):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    page_url = f"https://chinhsachonline.chinhphu.vn/bao-hiem-xa-hoi-bao-hiem-y-te-bao-hiem-that-nghiep/t1/1/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_bh = pd.DataFrame(articles_data)
df_bh.head(5)

,Name,Time,Question,Situation,Answer
0,Danh Ril,08:05 12/08/2024,Người dân tộc thiểu số xã nông thôn mới có đượ...,"Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã ...",Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/...
1,Ngô Đảm,09:05 02/08/2024,Có được truy lĩnh chế độ thai sản?,Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/201...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
2,Nguyễn Thảo,07:05 02/08/2024,Khi nào cần xin thêm Giấy chứng nhận nghỉ việc...,"Tôi xin hỏi, trên giấy ra viện có ghi chú ngườ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
3,Trần Thị Hạnh,08:45 29/07/2024,Đóng BHXH tự nguyện bao lâu thì được lương hưu?,"Tôi năm nay 53 tuổi, đóng BHXH tự nguyện được ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
4,Nguyễn Hòa Điều,13:33 18/07/2024,Đóng BHXH bao lâu trước khi sinh thì được hưởn...,Tôi đóng BHXH từ tháng 1/2020 đến tháng 12/202...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../data/data_tinhhuong/bhxh_bhyt_bhtn_data.csv'
df_bh.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../data/data_tinhhuong/bhxh_bhyt_bhtn_data.csv


In [6]:
df_bh=pd.read_csv("../data/data_tinhhuong/bhxh_bhyt_bhtn_data.csv",encoding='utf-8')

In [7]:
df_bh.head(5)

,Index,Name,Time,Question,Situation,Answer
0,0,Danh Ril,08:05 12/08/2024,Người dân tộc thiểu số xã nông thôn mới có đượ...,"Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã ...",Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/...
1,1,Ngô Đảm,09:05 02/08/2024,Có được truy lĩnh chế độ thai sản?,Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/201...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
2,2,Nguyễn Thảo,07:05 02/08/2024,Khi nào cần xin thêm Giấy chứng nhận nghỉ việc...,"Tôi xin hỏi, trên giấy ra viện có ghi chú ngườ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
3,3,Trần Thị Hạnh,08:45 29/07/2024,Đóng BHXH tự nguyện bao lâu thì được lương hưu?,"Tôi năm nay 53 tuổi, đóng BHXH tự nguyện được ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
4,4,Nguyễn Hòa Điều,13:33 18/07/2024,Đóng BHXH bao lâu trước khi sinh thì được hưởn...,Tôi đóng BHXH từ tháng 1/2020 đến tháng 12/202...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
